In [3]:
import pandas as pd
import re
from itertools import product, combinations
from fuzzywuzzy import fuzz, process

In [4]:
def clean_string(word):
    '''This function cleans the string text.
    It removes any special character, removes white
    spaces at the beginning or end or in between
    the text, removes any numeric values from
    the string and lastly converts the text
    into lower case'''
    
    word = word.strip()
    word = re.sub("[^A-Za-z]", "", word)
    word = word.lower()
    return word

### Defining lists of store names which have to be matched

In [5]:
a1 = ["Stop & Shop", "Walmart", "Trader Joe's", "Costco"]
a2 = ["TarderJoe", "Cost Co", "Walmrt", "Stop Shop"]
a3 = ["Stop n Shop of NYC ", "Walmart Store", "Traderjoes, NYC", "Costco Wholesale"]
a4 = ["Stop & Shop of Norwalk", "Walmart of Norwalk", "Trader Joe' s", "Costco of Norwalk"]

## Case: When data is present in a(n) array/list format and correct names are known

In [6]:
# the correct values are given in a "correct list" and this has to be compared to 
# another list containing variations of the correct strings

correct_list = a1
list_string = a2 + a3 + a4

In [7]:
clean_correct_list = [clean_string(x) for x in correct_list]
clean_list = [clean_string(x) for x in list_string]

In [8]:
def get_matches(correct, check_list):
    '''This function generates a dataframe
    containing the correct string names as the
    first column and any string which contains
    the correct names. The disadvantage of this
    method is that it check for an exact match
    of the correct names, e.g., the word dogs
    will not give a match to the word dog, but
    it will be correct the other way around.'''
    
    string1, string2 = [], []
    for x in correct:
        for y in check_list:
            if x in y:
                string1.append(x)
                string2.append(y)
    return pd.DataFrame(list(zip(string1, string2)), columns=['correct_name', 'matched_name'])

In [9]:
df_match = get_matches(clean_correct_list, clean_list)
df_match

,correct_name,matched_name
0,stopshop,stopshop
1,stopshop,stopshopofnorwalk
2,walmart,walmartstore
3,walmart,walmartofnorwalk
4,traderjoes,traderjoesnyc
5,traderjoes,traderjoes
6,costco,costco
7,costco,costcowholesale
8,costco,costcoofnorwalk


In [10]:
# the datframe returns the matched names where the score is >= 60.

d = {} # initialize an empty dictionary

for x in clean_correct_list:
    d[x] = [x[0] for x in process.extract(x, clean_list) if x[1] >= 60]

df_match = pd.DataFrame.from_dict(d, orient='index').T.unstack().dropna().reset_index(level=1,drop=True)
df_match

stopshop               stopshop
stopshop      stopshopofnorwalk
stopshop         stopnshopofnyc
walmart                  walmrt
walmart            walmartstore
walmart        walmartofnorwalk
traderjoes           traderjoes
traderjoes        traderjoesnyc
traderjoes            tarderjoe
costco                   costco
costco          costcowholesale
costco          costcoofnorwalk
dtype: object

## Case: When data is present in a(n) array/list format and the correct names are unknown

In [11]:
# creating a list of all the entries from the above 4 lists

list_string = a1 + a2 + a3 + a4
list_string

['Stop & Shop',
 'Walmart',
 "Trader Joe's",
 'Costco',
 'TarderJoe',
 'Cost Co',
 'Walmrt',
 'Stop Shop',
 'Stop n Shop of NYC ',
 'Walmart Store',
 'Traderjoes, NYC',
 'Costco Wholesale',
 'Stop & Shop of Norwalk',
 'Walmart of Norwalk',
 "Trader Joe' s",
 'Costco of Norwalk']

In [12]:
# cleaning the text in the given list

clean_list = [clean_string(x) for x in list_string]
len(clean_list)

16

In [13]:
def get_match_ratios(string_list):
    '''This function will create a dataframe
    containing all possible pairs of combinations
    of strings from all the cleaned columns, i.e.,
    if there are 2 columns with 2 entries, all 
    possible combinations will be a total of 4. 
    Then these pairs are checked to see how close
    of a match they are to each other, using the
    Levenshtein distance. The column fuzz_ratio
    gives the match score out of 100'''
    
    string1, string2, ratio = [], [], []
    
    # create all possible pairs of string text and get the fuzzy match score
    for x in combinations(string_list, 2):
        string1.append(x[0])
        string2.append(x[1])
        ratio.append(fuzz.ratio(x[0], x[1]))
    return pd.DataFrame(list(zip(string1, string2, ratio)), columns=['string1', 'string2', 'fuzz_ratio'])

In [14]:
df_match = get_match_ratios(clean_list)
df_match

,string1,string2,fuzz_ratio
0,stopshop,walmart,13
1,stopshop,traderjoes,11
2,stopshop,costco,43
3,stopshop,tarderjoe,24
4,stopshop,costco,43
...,...,...,...
115,stopshopofnorwalk,traderjoes,7
116,stopshopofnorwalk,costcoofnorwalk,75
117,walmartofnorwalk,traderjoes,23
118,walmartofnorwalk,costcoofnorwalk,65


In [15]:
# filtering out only those records for which the score >= 60
df_match[df_match['fuzz_ratio'] >= 60]

,string1,string2,fuzz_ratio
6,stopshop,stopshop,100
7,stopshop,stopnshopofnyc,73
11,stopshop,stopshopofnorwalk,64
19,walmart,walmrt,92
22,walmart,walmartstore,74
26,walmart,walmartofnorwalk,61
30,traderjoes,tarderjoe,84
36,traderjoes,traderjoesnyc,87
40,traderjoes,traderjoes,100
43,costco,costco,100


## Case: When the data is present in a dataframe and the correct names are unknown

In [16]:
# creating a dataframe out of the 4 lists above

df1 = pd.DataFrame(list(zip(a1, a2, a3, a4)), columns= ['col1', 'col2', 'col3', 'col4'])
df1

,col1,col2,col3,col4
0,Stop & Shop,TarderJoe,Stop n Shop of NYC,Stop & Shop of Norwalk
1,Walmart,Cost Co,Walmart Store,Walmart of Norwalk
2,Trader Joe's,Walmrt,"Traderjoes, NYC",Trader Joe' s
3,Costco,Stop Shop,Costco Wholesale,Costco of Norwalk


In [17]:
# cleaning the text in the dataframe columns and creatimng new columns with the cleaned text

for col in df1.columns:
    df1['new_'+col] = df1[col].apply(lambda x: clean_string(x))
df1

,col1,col2,col3,col4,new_col1,new_col2,new_col3,new_col4
0,Stop & Shop,TarderJoe,Stop n Shop of NYC,Stop & Shop of Norwalk,stopshop,tarderjoe,stopnshopofnyc,stopshopofnorwalk
1,Walmart,Cost Co,Walmart Store,Walmart of Norwalk,walmart,costco,walmartstore,walmartofnorwalk
2,Trader Joe's,Walmrt,"Traderjoes, NYC",Trader Joe' s,traderjoes,walmrt,traderjoesnyc,traderjoes
3,Costco,Stop Shop,Costco Wholesale,Costco of Norwalk,costco,stopshop,costcowholesale,costcoofnorwalk


In [18]:
# creating a list of pairings of cleaned columns from the dataframe

cols_list = list(combinations(df1.filter(regex='new').columns, 2))
cols_list

[('new_col1', 'new_col2'),
 ('new_col1', 'new_col3'),
 ('new_col1', 'new_col4'),
 ('new_col2', 'new_col3'),
 ('new_col2', 'new_col4'),
 ('new_col3', 'new_col4')]

In [19]:
def get_match_ratios(df, df_cols):
    '''This function will create a dataframe
    containing all possible pairs of combinations
    of strings from all the cleaned columns, i.e.,
    if there are 2 columns with 2 entries, all 
    possible combinations will be a total of 4. 
    Then these pairs are checked to see how close
    of a match they are to each other, using the
    Levenshtein distance. The column fuzz_ratio
    gives the match score out of 100'''
    
    string1, string2, ratio = [], [], []
    for col in df_cols:
        for x in product(df[col[0]], df[col[1]]):
            
            string1.append(x[0])
            string2.append(x[1])
            ratio.append(fuzz.ratio(x[0], x[1]))
    return pd.DataFrame(list(zip(string1, string2, ratio)), columns=['string1', 'string2', 'fuzz_ratio'])

In [20]:
df_match = get_match_ratios(df1, cols_list)
df_match

,string1,string2,fuzz_ratio
0,stopshop,tarderjoe,24
1,stopshop,costco,43
2,stopshop,walmrt,14
3,stopshop,stopshop,100
4,walmart,tarderjoe,25
...,...,...,...
91,traderjoesnyc,costcoofnorwalk,21
92,costcowholesale,stopshopofnorwalk,44
93,costcowholesale,walmartofnorwalk,19
94,costcowholesale,traderjoes,24


In [21]:
# filtering out only those records for which the score >= 60
df_match[df_match['fuzz_ratio'] >= 60]

,string1,string2,fuzz_ratio
3,stopshop,stopshop,100
6,walmart,walmrt,92
8,traderjoes,tarderjoe,84
13,costco,costco,100
16,stopshop,stopnshopofnyc,73
21,walmart,walmartstore,74
26,traderjoes,traderjoesnyc,87
32,stopshop,stopshopofnorwalk,64
37,walmart,walmartofnorwalk,61
42,traderjoes,traderjoes,100
